In [ ]:
import netsquid as ns
# Set qstate formalism to use density matrices.
ns.set_qstate_formalism(ns.QFormalism.DM)

# Exercise 1 Solution


In [ ]:
from netsquid.nodes import Node
from netsquid.components import Port, QuantumChannel
from netsquid.nodes import DirectConnection
from netsquid.components.models import FixedDelayModel, DepolarNoiseModel, DephaseNoiseModel

alice_node = Node("Alice")
bob_node   = Node("Bob")

alice_port = Port("Alice_port", alice_node)
bob_port   = Port("Bob_port",   bob_node)

delay_model = FixedDelayModel(delay=10.)
qerror_model = DephaseNoiseModel(dephase_rate=0.20, time_independent=True)

channel_a_to_b = QuantumChannel("Channel Alice to Bob",
                                  models={"delay_model": delay_model,
                                          "quantum_noise_model": qerror_model}
                                )

channel_b_to_a = QuantumChannel("Channel Bob to Alice",
                                  models={"delay_model": delay_model,
                                          "quantum_noise_model": qerror_model}
                                  )


connection = DirectConnection(name="connection",
                              channel_AtoB=channel_a_to_b,
                              channel_BtoA=channel_b_to_a)


alice_node.connect_to(remote_node=bob_node, connection=connection,
                     local_port_name="Alice_port", remote_port_name="Bob_port")

In [ ]:
from netsquid.protocols import NodeProtocol

class AliceProtocol(NodeProtocol):
    def run(self):
        port = self.node.ports["Alice_port"]

        q1, q2 = ns.qubits.create_qubits(2)
        
        ns.qubits.operate(q1, ns.H)
        ns.qubits.operate([q1, q2], ns.CX)
        
        port.tx_output(q2)
        print(f"{ns.sim_time()} ns Alice send q2")

        # Wait 30 ns
        yield self.await_timer(30)

        m, _ = ns.qubits.measure(q1)

        print(f"{ns.sim_time()} ns Alice measures q1: {m}")



class BobProtocol(NodeProtocol):
    def run(self):
        port = self.node.ports["Bob_port"]

        yield self.await_port_input(port)
        
        q2 = port.rx_input().items[0]

        dm = ns.qubits.reduced_dm(q2.qstate.qubits)
        print(f"{ns.sim_time()} ns Bob receives q2, DM=\n{dm}")

        m, _ = ns.qubits.measure(q2)

        print(f"{ns.sim_time()} ns Bob measures q2: {m}")

alice_protocol = AliceProtocol(node=alice_node)
bob_protocol = BobProtocol(node=bob_node)

In [ ]:
# Start all protocols
alice_protocol.start()
bob_protocol.start()

sim_stats = ns.sim_run()

print(sim_stats)

# Reset simulation
alice_protocol.stop()
bob_protocol.stop()

ns.sim_reset()

## Optional Exercise solution

In [ ]:
from netsquid.protocols import NodeProtocol

class AliceProtocol(NodeProtocol):
    def run(self):
        port = self.node.ports["Alice_port"]

        q1, q2 = ns.qubits.create_qubits(2)
        
        ns.qubits.operate(q1, ns.H)
        ns.qubits.operate([q1, q2], ns.CX)
        
        port.tx_output(q2)
        print(f"{ns.sim_time()} ns Alice send q2")

        yield self.await_port_input(port)
        q2 = port.rx_input().items[0]
        print(f"{ns.sim_time()} ns Alice received q2, \nDM:\n{ns.qubits.reduced_dm([q1, q2])}")

        m1, _ = ns.qubits.measure(q1)
        m2, _ = ns.qubits.measure(q2)

        print(f"{ns.sim_time()} ns Alice measures q1: {m1} and q2: {m2}")

        if m1 == m2:
            print("Alice concludes secret bit is 0")
        else:
            print("Alice concludes secret bit is 1")



class BobProtocol(NodeProtocol):
    def __init__(self, node, secret_bit):
        super().__init__(node)
        self.secret_bit = secret_bit
    
    def run(self):
        port = self.node.ports["Bob_port"]

        yield self.await_port_input(port)
        
        q2 = port.rx_input().items[0]
        print(f"{ns.sim_time()} ns Bob receives q2")
        if self.secret_bit == 1:
            ns.qubits.operate(q2, ns.X)

        port.tx_output(q2)


In [ ]:
ns.sim_reset()
alice_port.reset()
bob_port.reset()
connection.reset()

secret_bit = 1

alice_protocol = AliceProtocol(node=alice_node)
bob_protocol = BobProtocol(node=bob_node, secret_bit=secret_bit)

# Start all protocols
alice_protocol.start()
bob_protocol.start()

sim_stats = ns.sim_run()

print(sim_stats)

# Reset simulation
alice_protocol.stop()
bob_protocol.stop()